<img src="./assets/airbnb_logo.png"
    style="width:300px; float: right; margin: 0 40px 40px 40px;"></img>
# Group Project AirBnB
**Useful links**

<a href="https://github.com/ShimantoRahman/aCRM-Group-Project-AirBnB"><img src="./assets/github_logo.png" style="width:120px; margin: 0 0 40px 40px;"></a>

> [Inside AirBnB: New York](http://insideairbnb.com/new-york-city/?fbclid=IwAR3lvDyNFboZqns1jNJ8v4OzqzG8sLFsqeSlRjqb_-tyvk4iM_XRSYdwmEQ)

> [Airbnb Rental Listings Dataset Mining](https://towardsdatascience.com/airbnb-rental-listings-dataset-mining-f972ed08ddec)

## 1 Setup
### 1.1 Import modules

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from sklearn.preprocessing import LabelEncoder
# nltk.download('punkt')
# nltk.download('stopwords')

![green-divider](./assets/green_divider.png)

### 1.2 Read data

In [ ]:
calendar_detail = pd.read_csv("./data/calendar_detail.csv")
listings_summary = pd.read_csv("./data/listings_summary.csv")
reviews_summary = pd.read_csv("./data/reviews_summary.csv")
neighbourhoods = pd.read_csv("./data/neighbourhoods.csv")

In [ ]:
listings_detail = pd.read_csv("./data/listings_detail.csv")

In [ ]:
reviews_detail = pd.read_csv("./data/reviews_detail.csv")

In [ ]:
calendar_detail.head()

In [ ]:
listings_summary.head()

In [ ]:
reviews_summary.head()

In [ ]:
neighbourhoods.head()

In [ ]:
listings_detail.head()

In [ ]:
reviews_detail.head()

![green-divider](./assets/green_divider.png)

### 1.3 data preparation
#### 1.3.1 Detecting NaN values

In [ ]:
print("listings_summary\n")
print(listings_summary.isnull().sum())

In [ ]:
print("listings_detail\n")
print(listings_detail.isnull().sum())

In [ ]:
print("reviews_summary\n")
print(reviews_summary.isnull().sum())

In [ ]:
print("reviews_detail\n")
print(reviews_detail.isnull().sum())

In [ ]:
print("neighbourhoods\n")
print(neighbourhoods.isnull().sum())

In [ ]:
print("\ncalendar_detail\n")
print(calendar_detail.isnull().sum())

![green-divider](./assets/green_divider.png)

#### 1.3.2 Cleansing data

In [ ]:
# removing listings where first and last review do not both match
listings_detail = listings_detail[~((listings_detail["first_review"].isnull()) & ~(listings_detail["last_review"].isnull()))]

In [ ]:
# replacing NaN values for reviews_per_month to 0
# rows with NaN value for reviews_per_month do not have a first or last review, thus they have 0 reviews per month
column_imputations = {"reviews_per_month": 0}
listings_detail = listings_detail.fillna(value = column_imputations)

![green-divider](./assets/green_divider.png)

#### 1.3.2 Correcting data types

In [ ]:
# dates
def column_to_date(df, column):
    df[column] = pd.to_datetime(df[column], format="%Y-%m-%d")
    
# listings_summary
column_to_date(listings_summary, "last_review")

# reviews_summary
column_to_date(reviews_summary, "date")

# calendar_detail
column_to_date(calendar_detail, "date")

# listings_detail
column_to_date(listings_detail, "first_review")
column_to_date(listings_detail, "last_review")
column_to_date(listings_detail, "last_scraped")
column_to_date(listings_detail, "calendar_last_scraped")
column_to_date(listings_detail, "host_since")

# reviews_detail

In [ ]:
# change t/f columns to 1/0
label_encoder = LabelEncoder()

def column_to_bool(df, column):
    label_encoder.fit(df[column])
    new_column = column + "_num"
    df[new_column] = label_encoder.transform(df[column])

column_to_bool(listings_detail, "instant_bookable")
column_to_bool(listings_detail, "requires_license")
column_to_bool(listings_detail, "is_business_travel_ready")
column_to_bool(listings_detail, "require_guest_profile_picture")
column_to_bool(listings_detail, "require_guest_phone_verification")
column_to_bool(listings_detail, "instant_bookable")
# column_to_bool(listings_detail, "host_is_superhost")
column_to_bool(listings_detail, "has_availability")
listings_detail[["requires_license", "requires_license_num"]].head()

In [ ]:
# changing price from $00.0 [String] to 00.0 float 

calendar_detail["price"] = calendar_detail["price"].str.replace("$", "")
calendar_detail["price"] = calendar_detail["price"].str.replace(",", "")
calendar_detail["price"] = pd.to_numeric(calendar_detail["price"])

![green-divider](./assets/green_divider.png)

## 2 Analysis
### 2.1 Calculate the average listing price per neighbourhood

In [ ]:
print(listings_summary.groupby("neighbourhood").mean()["price"])

### 2.2 Plot how the average price evolves through the year across New York.

In [ ]:
calendar_detail[["date", "price"]].groupby("date").mean().plot()

### 2.3	Identify which neighborhood has the largest price fluctuations across the year. Plot the fluctuations for this neighborhood.

In [ ]:
# joining listings_summary with calendar_detail
temp = listings_summary[["id", "neighbourhood"]].rename(columns = {"id": "listing_id"})
cal_listing = pd.merge(calendar_detail, temp, how="inner", on="listing_id")
cal_listing.head()

In [ ]:
# calculating variance per neighbourhood and requesting top one
price_by_neighbourhood = cal_listing[["neighbourhood", "price"]].groupby("neighbourhood").var()
price_by_neighbourhood.sort_values(by = "price", ascending = False).head(1)

In [ ]:
# plotting the prices of the neighbourhood with the largest price fluctuations
cal_listing[["date", "price", "neighbourhood"]][cal_listing["neighbourhood"] == "Randall Manor"].groupby("date").mean().plot()

### 2.4 In marketing, there is a phenomenon known as ‘the long tail’ (Hint: look it up). This also translates to the number of reviews. Plot this on an intuitive graph.

### 2.5	Run a regression to explain the price per listing. (Hint: location, reviews, etc. may all explain this).

### 2.6 Find additional data sources to explain the average listing price per neighbourhood. (Hint : think demographics)

### 2.7 Plot how the average prices differ across New York using a color-coded heat map of New York neighborhoods.

### 2.8 The latitude of Statue of Liberty National Monument, New York, USA is 40.68927, and the longitude is -74.044502. This monument is one of the most popular tourist places in New York. Statistically test wether a distance smaller than 2 miles to the monument increases average listing price.

### 2.9	Create a timeline and plot for each month the highest, Q1, the median, Q3 and lowest price on one graph. Do this for each neighborhood group as well as for the entire city. Determine which neighborhood group stands out the most and create a comparative graph of this neighborhood with all other groups.

### 2.10 Plot the number of rooms per host in function of the number of reviews per host. 

### 2.11 Are there a lot of hosts having multiple locations? Do most people just rent their own place? Is there a ‘host long tail’? Make a comprehensive plot.

### 2.12 Do hosts with multiple locations stay within the same neighbourhood? (hint: use subset)

In [ ]:
# subset of hosts with multiple locations
ids = listings_summary["host_id"]
subset = listings_summary[ids.isin(ids[ids.duplicated()])].sort_values(by="host_id")

# count hosts with different locations and hosts with only 1 location
more_than_one_loc = 0
just_one_loc = 0
for i in ids[ids.duplicated()]:
    if len(subset[subset["host_id"] == i]["neighbourhood"].unique()) > 1:
        more_than_one_loc += 1
    else:
        just_one_loc += 1
        
print("more than one location: %s" %more_than_one_loc) # 1201
print("just one location: %s" %just_one_loc) # 6008

In [ ]:
# bar chart of counts


### 2.13 What are the 5 most used words in reviews that are no stop words? (e.g. the, or, etc. Python can filter these automatically using packages such as NLTK).

### 2.14 Do these most frequent words differ across neighborhoods? What are the ‘most different’ areas? What distinguishes them? Interpret.

### 2.15 Plot the amount of reviews across time. 

### 2.16 Is there a link between availability (days per year) with the price? Determine both graphically and statistically. 

### 2.17 Is there a link between how many times the word ‘great’ appears in a review and the listing price? Determine both graphically and statistically. 

### 2.18 Plot how the number of Airbnb locations are distributed across the city on a map. Plot the number of locations per neighborhood and color code according to neighborhood group.

### 2.19 Williamsburg is a ‘hip’ area in in Brooklyn with a lot of Airbnb locations on offer. Explore how this area differs from other locations and visualize. You may also use external data sources.

### 2.20  Create a stacked bar chart of the distribution of room type per neighborhood group. Statistically test whether these differences are significant.

### 2.21 Color-coded plot the most popular room type per neighborhood on a city map.